Day 5 Assessment: Airline Pricing Data Analysis

Task 1: Data Generation

I am generating a synthetic dataset of 320 airline tickets. I use a specific seed_value and fixed formulas to ensure the dataset is reproducible. I am also intentionally injecting "dirty" data (missing prices, negative prices, and casing inconsistencies) to simulate real-world data cleaning challenges.

In [1]:
import numpy as np
import json

# 1. SETUP & SEEDING
seed_value = 2609 
n = 320

# Initializing the Generator
rng = np.random.default_rng(seed_value)

# 2. GENERATION LOGIC
tickets = []
routes = ["NYC-LAX", "LHR-JFK", "SFO-SEA", "DXB-SIN", "MAD-ROM"]
days_list = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
classes = ["economy", "premium", "business"]
route_adjs = [140, 220, 60, 180, 80]
class_adjs = [0, 80, 220]

# Loop from 1 to n (inclusive) as per instructions
for i in range(1, n + 1):
    # Determining indices using the provided formulas
    route_idx = (i + seed_value) % 5
    day_idx = (i + seed_value) % 7
    class_idx = (i * 2 + seed_value) % 3
    
    # Calculating days to departure
    dtd = 1 + ((i * 3 + seed_value) % 60)
    
    # Calculating base price and adjustments
    base = 120 + (dtd * -1.5)
    noise = rng.normal(0, 25)
    price = round(base + route_adjs[route_idx] + class_adjs[class_idx] + noise, 2)
    
    # Defining the core record
    ticket = {
        "ticket_id": f"T{seed_value}-{i:04d}", # :04d pads with zeros (e.g., 0001)
        "route": routes[route_idx],
        "day": days_list[day_idx],
        "days_to_departure": dtd,
        "class": classes[class_idx],
        "price_usd": price
    }
    
    # 3. INJECTING DATA ISSUES (Deterministic Errors)
    # If i % 28 == 0, set price to empty string
    if i % 28 == 0:
        ticket["price_usd"] = ""
        
    # If i % 45 == 0, make price negative
    if i % 45 == 0:
        ticket["price_usd"] = ticket["price_usd"] * -1 if isinstance(ticket["price_usd"], (int, float)) else ticket["price_usd"]
        
    # If i % 37 == 0, make class uppercase
    if i % 37 == 0:
        ticket["class"] = ticket["class"].upper()
        
    tickets.append(ticket)

# Final check for Task 1
print(f"Total records generated: {len(tickets)}")
print("\nFirst 5 entries:")
print(json.dumps(tickets[:5], indent=2))

Total records generated: 320

First 5 entries:
[
  {
    "ticket_id": "T2609-0001",
    "route": "NYC-LAX",
    "day": "Sun",
    "days_to_departure": 33,
    "class": "premium",
    "price_usd": 288.12
  },
  {
    "ticket_id": "T2609-0002",
    "route": "LHR-JFK",
    "day": "Mon",
    "days_to_departure": 36,
    "class": "economy",
    "price_usd": 308.74
  },
  {
    "ticket_id": "T2609-0003",
    "route": "SFO-SEA",
    "day": "Tue",
    "days_to_departure": 39,
    "class": "business",
    "price_usd": 339.46
  },
  {
    "ticket_id": "T2609-0004",
    "route": "DXB-SIN",
    "day": "Wed",
    "days_to_departure": 42,
    "class": "premium",
    "price_usd": 256.2
  },
  {
    "ticket_id": "T2609-0005",
    "route": "MAD-ROM",
    "day": "Thu",
    "days_to_departure": 45,
    "class": "economy",
    "price_usd": 137.19
  }
]


Task 2: Data Validation and Cleaning

I am filtering the tickets list to remove records with missing or negative prices. I am also normalizing the class field by converting all strings to lowercase to ensure consistency for future analysis.

In [2]:
cleaned_tickets = []

for t in tickets:
    price = t["price_usd"]
    
    # 1. VALIDATION CHECKS
    # Checking if price is a number (not an empty string)
    is_numeric = isinstance(price, (int, float))
    
    # Checking if price is positive
    is_positive = False
    if is_numeric:
        if price > 0:
            is_positive = True
            
    # 2. CLEANING / NORMALIZATION
    if is_numeric and is_positive:
        # Creating a copy to avoid modifying the original list
        clean_t = t.copy()
        # Normalizing class to lowercase
        clean_t["class"] = clean_t["class"].lower()
        cleaned_tickets.append(clean_t)

# Confirming results
print(f"Original Count: {len(tickets)}")
print(f"Cleaned Count:  {len(cleaned_tickets)}")
print(f"Removed:        {len(tickets) - len(cleaned_tickets)} invalid records")

print("\nExample of cleaned records (confirming lowercase class):")
print(cleaned_tickets[:2])

Original Count: 320
Cleaned Count:  302
Removed:        18 invalid records

Example of cleaned records (confirming lowercase class):
[{'ticket_id': 'T2609-0001', 'route': 'NYC-LAX', 'day': 'Sun', 'days_to_departure': 33, 'class': 'premium', 'price_usd': 288.12}, {'ticket_id': 'T2609-0002', 'route': 'LHR-JFK', 'day': 'Mon', 'days_to_departure': 36, 'class': 'economy', 'price_usd': 308.74}]


Task 3: NumPy Vectorized Analysis

I am converting the cleaned list of dictionaries into NumPy arrays. This allows for high-performance mathematical operations. I will compute the overall statistics and then use Boolean Masking to calculate revenue and ticket counts for each day of the week without using loops.

In [3]:
# 1. Creating arrays from the cleaned list
# We use list comprehension to "pluck" specific values out of the dictionaries
prices = np.array([t["price_usd"] for t in cleaned_tickets])
days = np.array([t["day"] for t in cleaned_tickets])

# 2. Basic Statistics
overall_mean = np.mean(prices)
overall_std = np.std(prices)

# 3. Daily Revenue and Counts (Vectorized)
# We define the order to ensure our results are organized
days_order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
daily_totals = {}
daily_counts = {}

for d in days_order:
    # BOOLEAN MASK: Creating a True/False array where day matches 'd'
    mask = (days == d)
    
    # Using the mask to select only prices for that specific day
    daily_totals[d] = np.sum(prices[mask])
    daily_counts[d] = np.sum(mask)

# 4. Validation Check
# Does the sum of daily revenues match the total price array?
total_revenue_from_prices = np.sum(prices)
total_revenue_from_days = sum(daily_totals.values())
revenue_match = np.isclose(total_revenue_from_prices, total_revenue_from_days)

print(f"Overall Mean Price: ${overall_mean:.2f}")
print(f"Overall Std Dev:    ${overall_std:.2f}")
print(f"Revenue Match Check: {revenue_match}")

Overall Mean Price: $308.02
Overall Std Dev:    $111.94
Revenue Match Check: True


Task 4: High-Price Ticket Identification

I am identifying the 90th percentile threshold for ticket prices. Any ticket at or above this value is considered "High-Price." I will then verify that my filtering logic correctly captured only tickets meeting this criterion.

In [4]:
# 1. Computing the 90th percentile threshold
threshold = np.percentile(prices, 90)

# 2. Creating a mask for prices >= threshold
high_price_mask = prices >= threshold
high_price_tickets = prices[high_price_mask]

# 3. Validation
# The count should be roughly 10% of our cleaned data
high_price_count = len(high_price_tickets)
all_above_threshold = np.all(high_price_tickets >= threshold)

print(f"90th Percentile Threshold: ${threshold:.2f}")
print(f"Number of High-Price Tickets: {high_price_count}")
print(f"Validation - All selected are >= threshold: {all_above_threshold}")

90th Percentile Threshold: $464.33
Number of High-Price Tickets: 31
Validation - All selected are >= threshold: True


Task 5: Final Summary Report

This final section aggregates all computed metrics into a structured report. I am including explicit validation statements to confirm that the data cleaning and analysis process remained consistent throughout the workflow.

In [5]:
# 1. Building the report dictionary
report = {
    "total_tickets_generated": len(tickets),
    "cleaned_tickets_count": len(cleaned_tickets),
    "mean_ticket_price": round(overall_mean, 2),
    "std_dev_price": round(overall_std, 2),
    "daily_revenue_totals": {k: round(v, 2) for k, v in daily_totals.items()},
    "high_price_ticket_count": high_price_count
}

# 2. Printing formatted report
print("---------- FINAL AIRLINE PRICING REPORT ----------")
print(json.dumps(report, indent=4))

# 3. Final Explicit Validation Statement
# Confirming that cleaned_tickets is <= total_tickets
validation_pass = len(cleaned_tickets) <= len(tickets)

print("\n--- FINAL VALIDATION ---")
print(f"Data Integrity Check (Cleaned <= Total): {validation_pass}")
print(f"Total Revenue Reconciled: {revenue_match}")
print("--------------------------------------------------")

---------- FINAL AIRLINE PRICING REPORT ----------
{
    "total_tickets_generated": 320,
    "cleaned_tickets_count": 302,
    "mean_ticket_price": 308.02,
    "std_dev_price": 111.94,
    "daily_revenue_totals": {
        "Mon": 13762.33,
        "Tue": 13660.19,
        "Wed": 13898.88,
        "Thu": 13831.6,
        "Fri": 13448.02,
        "Sat": 10542.51,
        "Sun": 13878.88
    },
    "high_price_ticket_count": 31
}

--- FINAL VALIDATION ---
Data Integrity Check (Cleaned <= Total): True
Total Revenue Reconciled: True
--------------------------------------------------
